In [3]:
import pandas as pd
import datetime
import vectorbt as vbt
import numpy as np

c:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [9]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 1, )
eth_price = vbt.BinanceData.download(['ETHUSDT', 'BTCUSDT'],
                                interval = '1m',
                                start = start_date,
                                end = end_date,
                                missing_index= 'drop').get("Close")

eth_price


0it [00:00, ?it/s]

0it [00:00, ?it/s]

symbol,ETHUSDT,BTCUSDT
Open time,,
2023-07-13 14:50:00+00:00,1882.74,30583.00
2023-07-13 14:51:00+00:00,1882.80,30576.00
2023-07-13 14:52:00+00:00,1882.33,30575.67
2023-07-13 14:53:00+00:00,1882.59,30580.00
2023-07-13 14:54:00+00:00,1882.12,30570.23
...,...,...
2023-07-14 14:45:00+00:00,1995.87,31223.99
2023-07-14 14:46:00+00:00,1995.43,31215.00
2023-07-14 14:47:00+00:00,1996.21,31232.51


In [10]:
fast_ma = vbt.MA.run(eth_price, 10)
slow_ma = vbt.MA.run(eth_price, 50)
rsi = vbt.RSI.run(eth_price)

In [11]:
entries = fast_ma.ma_crossed_above(slow_ma) & rsi.rsi_above(50)
exits = slow_ma.ma_crossed_above(fast_ma) & rsi.rsi_below(50)

In [12]:
pf = vbt.Portfolio.from_signals(eth_price, entries, exits, init_cash=10000)
pf.total_return()

ma_window  ma_window  symbol 
50         10         ETHUSDT    0.011174
                      BTCUSDT    0.007730
Name: total_return, dtype: float64

# Indicator Factory

In [15]:
def custom_indicator(close, rsi_window = 14, ma_window = 50, entry = 30, exit = 70):
    close_5m = close.resample('5T').last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    rsi, _ = rsi.align(close, broadcast_axis = 0, method = 'ffill', join = 'right')
    
    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > exit, -1, 0)
    trend = np.where ((rsi < entry) & (close < ma), 1, trend)
    return trend
    


In [21]:
res

In [17]:
ind = vbt.IndicatorFactory(
        class_name= 'combination',
        short_name = 'comb',
        input_names = ["close" ],
        param_names = ["rsi_window", "ma_window", "entry", "exit"],
        output_names= ["value"]
    ).from_apply_func(custom_indicator,
                      rsi_window = 14,
                      ma_window = 50,
                      entry = 30,
                      exit = 70,
                      keep_pd = True)

res = ind.run(eth_price, 
              rsi_window = np.arange(10, 40, step = 3, dtype = int),
              ma_window = np.arange(20, 200, step = 15, dtype = int),
              entry = np.arange(10, 40, step = 4, dtype = int),
              exit = np.arange(60, 85, step = 4, dtype = int),
              param_product = True
              )


In [19]:
eth_price

symbol,ETHUSDT,BTCUSDT
Open time,,
2023-07-13 14:50:00+00:00,1882.74,30583.00
2023-07-13 14:51:00+00:00,1882.80,30576.00
2023-07-13 14:52:00+00:00,1882.33,30575.67
2023-07-13 14:53:00+00:00,1882.59,30580.00
2023-07-13 14:54:00+00:00,1882.12,30570.23
...,...,...
2023-07-14 14:45:00+00:00,1995.87,31223.99
2023-07-14 14:46:00+00:00,1995.43,31215.00
2023-07-14 14:47:00+00:00,1996.21,31232.51


In [23]:
res.output_names


('entries', 'exits')

In [ ]:
entries = res.value == 1.0
exits = res.value == -1.0


In [13]:
pf = vbt.Portfolio.from_signals(eth_price, entries, exits)
returns = pf.total_return()

returns_eth = returns[returns.index.isin(["ETH-USD"], level = 'symbol')]
print(returns.max())
print(returns.idxmax())
returns_eth

0.011174480558999278
(50, 10, 'ETHUSDT')


Series([], Name: total_return, dtype: float64)

In [ ]:
fig = returns.vbt.heatmap(x_level = "comb_rsi_window",
                          y_level = "comb_ma_window",
                          slider_level = 'symbol')
fig.show()